In [36]:
import numpy as np

**Прямые методы**
1) Метод Гаусса (Метод единичного деления)

In [37]:
def gauss_elimination(A, b):
    n = len(b)
    for k in range(n-1):
        for i in range(k+1, n):
            factor = A[i,k] / A[k,k]
            for j in range(k, n):
                A[i,j] -= factor * A[k,j]
            b[i] -= factor * b[k]
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (b[i] - np.dot(A[i,i+1:], x[i+1:])) / A[i,i]
    return x


2) Метод исключения (прямоугольника)

In [38]:
def rectangular_elimination(A, b):
    n = len(b)
    for k in range(n-1):
        for i in range(k+1, n):
            factor = A[i,k] / A[k,k]
            A[i] = A[i] - factor * A[k]
            b[i] = b[i] - factor * b[k]
    x = np.linalg.solve(A, b)
    return x


3) Выбор ведущего элемента по столбцу

In [39]:
def partial_pivoting_elimination(A, b):
    n = len(b)
    
    for k in range(n-1):
        # Выбор ведущего элемента по столбцу
        max_row_index = np.argmax(np.abs(A[k:, k])) + k
        A[[k, max_row_index]] = A[[max_row_index, k]]
        b[k], b[max_row_index] = b[max_row_index], b[k]
        
        for i in range(k+1, n):
            factor = A[i, k] / A[k, k]
            A[i, k:] -= factor * A[k, k:]
            b[i] -= factor * b[k]
            
    # Обратный ход
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]
        
    return x

4) Метод прогонки

In [40]:
def tridiagonal_solve(a, b, c, d):
    n = len(d)
    c_ = np.zeros(n-1)
    d_ = np.zeros(n)
    x = np.zeros(n)
    c_[0] = c[0] / b[0]
    d_[0] = d[0] / b[0]
    for i in range(1, n-1):
        c_[i] = c[i] / (b[i] - a[i-1]*c_[i-1])
    for i in range(1, n):
        d_[i] = (d[i] - a[i-1]*d_[i-1]) / (b[i] - a[i-1]*c_[i-1])
    x[-1] = d_[-1]
    for i in range(n-2, -1, -1):
        x[i] = d_[i] - c_[i] * x[i+1]
    return x

5) LU-разложение

In [41]:
def LU_decomposition(A):
    n = len(A)
    L = np.eye(n)
    U = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            U[i,j] = A[i,j] - sum(L[i,k] * U[k,j] for k in range(i))
            if i != j:
                L[j,i] = (A[j,i] - sum(L[j,k] * U[k,i] for k in range(i))) / U[i,i]
    return L, U

def LU_solve(L, U, b):
    n = len(b)
    y = np.zeros(n)
    x = np.zeros(n)
    for i in range(n):
        y[i] = b[i] - sum(L[i,j] * y[j] for j in range(i))
    for i in range(n-1, -1, -1):
        x[i] = (y[i] - sum(U[i,j] * x[j] for j in range(i+1, n))) / U[i,i]
    return x

**Итерационные методы**
1) Метод простых итераций

In [42]:
def simple_iterations(A, b, tol=1e-10, max_iter=1000):
    n = len(b)
    x = np.zeros(n)
    for _ in range(max_iter):
        x_new = np.dot(np.linalg.inv(A), b)
        if np.linalg.norm(x_new - x, np.inf) < tol:
            return x_new
        x = x_new
    return x

2) Метод Зейделя

In [43]:
def seidel(A, b, tol=1e-10, max_iter=1000):
    n = len(b)
    x = np.zeros(n)
    L = np.tril(A)
    U = A - L
    for _ in range(max_iter):
        x_new = np.dot(np.linalg.inv(L), b - np.dot(U, x))
        if np.linalg.norm(x_new - x, np.inf) < tol:
            return x_new
        x = x_new
    return x

**Тестирование**

In [44]:
A = np.array([[4, 1, 0], [1, 4, 1], [0, 1, 4]])
b = np.array([5, 6, 5])

print("Метод Гаусса:", gauss_elimination(A.copy(), b.copy()))
print("Метод исключения:", rectangular_elimination(A.copy(), b.copy()))
print("Выбор ведущего элемента по столбцу:", rectangular_elimination(A.copy(), b.copy()))

L, U = LU_decomposition(A.copy())
print("LU-разложение:", LU_solve(L, U, b.copy()))

print("Метод простых итераций:", simple_iterations(A.copy(), b.copy()))
print("Метод Зейделя:", seidel(A.copy(), b.copy()))

Метод Гаусса: [1. 1. 1.]
Метод исключения: [1. 1. 1.]
Выбор ведущего элемента по столбцу: [1. 1. 1.]
LU-разложение: [1. 1. 1.]
Метод простых итераций: [1. 1. 1.]
Метод Зейделя: [1. 1. 1.]
